In [1]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu
from transformers import T5ForConditionalGeneration, T5Tokenizer

nltk.download('punkt')

# Carga del dataset
df_ref = pd.read_csv("df_ref.csv")

model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

bleu_scores = []

def generate_summary_t5(article, reference_summary):
    input_text = "summarize: " + article
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    summary_ids = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    reference_tokens = nltk.word_tokenize(reference_summary)
    hypothesis_tokens = nltk.word_tokenize(generated_summary)
    bleu = sentence_bleu([reference_tokens], hypothesis_tokens)
    
    return generated_summary, bleu
e
for index, row in df_ref.iterrows():
    article = row['article']
    reference_summary = row['abstract']
    
    generated_summary, bleu = generate_summary_t5(article, reference_summary)
    
    bleu_scores.append(bleu)

bleu_scores_df = pd.DataFrame({
    'BLEU': bleu_scores
})

pd.options.display.float_format = '{:.10f}'.format

print(bleu_scores_df)


C:\Users\oscarandres.pinilla\AppData\Local\Temp\ipykernel_21356\2781410315.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\oscarandres.pinilla\anaconda3\envs\tfm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oscarandres.pinilla\AppData\Roaming\nltk_data
[nltk_data]     ...
[nltk_data]   Package punkt is already up-to-date!
You are using the defaul

           BLEU
0  0.0019906194
1  0.0000000000
2  0.0584887993
3  0.1207528166
4  0.0007681192
5  0.0179435640
6  0.0000000000
7  0.0012691017
8  0.0173535382
9  0.0329923109
10 0.0000000000
11 0.0822033535
12 0.0000000000
13 0.0002401373
14 0.1013743580
15 0.0000000000
16 0.0000000000
17 0.0574372412
18 0.0000000000
19 0.0000000000


In [2]:
bleu_scores_df

,BLEU
0,0.0019906194
1,0.0000000000
2,0.0584887993
3,0.1207528166
4,0.0007681192
5,0.0179435640
6,0.0000000000
7,0.0012691017
8,0.0173535382
9,0.0329923109


In [3]:
# Para posteriores cálculos se extrae el dataframe 
bleu_scores_df.to_csv('BLEU_scores_T5_df.csv', index=False)